In [1]:
cd E:/GitHub/computervision

E:\GitHub\computervision


In [2]:
import torch
import os
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

from torch.nn import CrossEntropyLoss
from datetime import datetime
from torch.optim import lr_scheduler, SGD, NAdam, Adam
from modules.training_util import PytorchDataset, PytorchTraining, EarlyStopping
from modules.models_util import ViT

In [3]:
# Variables for continuing previous training
CONTINUE_TRAINING_DIR = None
# CONTINUE_TRAINING_DIR = 'notebooks/vit/models/'

MODEL_WEIGHTS_PATH = None

if CONTINUE_TRAINING_DIR:
    MODEL_WEIGHTS_PATH = os.path.join(CONTINUE_TRAINING_DIR,'epoch_15.pth')

# CONFIG VARIABLES
FULLY_TRAIN = True
SAVE_WEIGHTS = True

base_path = 'E:/Datasets/cor-splits'
DATA_DIR = [
    # os.path.join(base_path,'sgkf-8-1-1-size-540'),
    # os.path.join(base_path,'underperforming_cameras_96'),
    # os.path.join(base_path,'underperforming_cameras_15'),
    # os.path.join(base_path,'underperforming_cameras_30'),
    # os.path.join(base_path,'underperforming_cameras_60_v2'),
    # os.path.join(base_path,'underperforming_cameras_61_15img'),
    os.path.join(base_path,'underperforming_cameras_59_45imgs'),
    # os.path.join(base_path,'underperforming_cameras_87_30imgs_poca'),
    # os.path.join(base_path,'random_cameras_61_30imgs'),
    # os.path.join(base_path,'random_cameras_59_45imgs'),
    # os.path.join(base_path,'random_cameras_71_15img'),
    # os.path.join(base_path,'underperforming_cameras_60_v2_euro'),
    # os.path.join(base_path,'underperforming_cameras_60_v2_itsc'),
    # os.path.join(base_path,'sgkf-8-2-size-1010'),
    # 'E:/Datasets/itsc_flood_dataset'
    ]
START_EPOCH = 1 # Change it from 1 if you're continuing an existing run
N_EPOCHS = 50
BATCH_SIZE = 16
SAVE_INTERVAL = 2 # How many epochs between each backup
## Learning Rate Scheduler
LR = [1e-03]  # When continuing a existing run, update it to the last epoch LR value
LR_EPOCH_SCHEDULE = 8  # Number of epochs before altering LR
LR_GAMMA = 0.1
## Early Stopping
PATIENCE = 10
MIN_DELTA = 0
MIN_EPOCH = 10 # Number of epochs before starting patience counter
##
BP = [0.5]

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device = {device}")
archs = ['b32']

Device = cuda:0


In [ ]:
for arch in archs:
    for dir in DATA_DIR:
        for lr in LR:
            for brightness in BP:
                vit = ViT(architecture=arch,bp=brightness)
                
                model = vit.load(MODEL_WEIGHTS_PATH, FULLY_TRAIN)
                model = model.to(device)
                
                criterion = CrossEntropyLoss()

                if FULLY_TRAIN:
                    params = model.parameters()
                else:
                    params = model.heads.parameters()
                    
                optimizer = SGD(params, lr=lr)
                # optimizer = Adam(params, lr, betas=(0.8,0.999),eps=1e-7)
                # optimizer = NAdam(params, lr, betas=(0.8,0.999),eps=1e-7)
                
                step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=LR_EPOCH_SCHEDULE, gamma=LR_GAMMA)
                es = EarlyStopping(patience=PATIENCE, delta=MIN_DELTA, min_epoch=MIN_EPOCH)
                dataset = PytorchDataset(dir, vit.data_transforms, BATCH_SIZE, num_workers=1)

                working_folder = 'notebooks/vit'
                models_folder = 'models'

                curr_time = datetime.now()
                curr_time = curr_time.strftime("%Y-%m-%d %H-%M-%S")

                output_folder = os.path.join(working_folder, models_folder)
                output_folder = os.path.join(output_folder, curr_time)
                
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)

                trainer = PytorchTraining(device, dataset, output_folder)
                model = trainer.train_pytorch_model(model, criterion, optimizer, vit, step_lr_scheduler, es, START_EPOCH, N_EPOCHS, SAVE_INTERVAL, SAVE_WEIGHTS)

All parameters for model VisionTransformer requires grad.
Saving model to folder notebooks/vit\models\2024-12-07 11-01-25

Model VisionTransformer b32
Fully Trained = True
Dataset E:/Datasets/cor-splits\underperforming_cameras_96
Learning Rate Epoch Schedule = 8                    
Learning Rate Gamma = 0.1
Optimizer = SGD
Batch Size = 16
Transforms = Compose(
    RandomHorizontalFlip(p=0.5)
    Resize(size=256, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485 0.456 0.406], std=[0.229 0.224 0.225])
)



Epoch 1/50
----------



c:\Users\Adm\anaconda3\envs\computervision\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


train Loss: 0.6283 Acc: 0.6471
val Loss: 0.5617 Acc: 0.6962
Epoch duration: 3 m 56s
Learning Rate = 0.001



Epoch 2/50
----------

train Loss: 0.4470 Acc: 0.8091
val Loss: 0.5827 Acc: 0.6667
Epoch duration: 2 m 2s
Learning Rate = 0.001



Epoch 3/50
----------

train Loss: 0.3056 Acc: 0.8838
val Loss: 0.5286 Acc: 0.7295
Epoch duration: 2 m 1s
Learning Rate = 0.001



Epoch 4/50
----------

train Loss: 0.2179 Acc: 0.9201
